In [2]:
#드라이브 마운트 - 구글 드라이브의 데이터를 가져올 때 가져오는 라이브러리
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#필요한 모듈 import
import pandas as pd
import numpy as np
import os

In [4]:
#data directory 확인
for dirname, _, filenames in os.walk('/content/drive/MyDrive/MoveLens/'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/MoveLens/genome_scores.csv
/content/drive/MyDrive/MoveLens/link.csv
/content/drive/MyDrive/MoveLens/movie.csv
/content/drive/MyDrive/MoveLens/rating.csv
/content/drive/MyDrive/MoveLens/genome_tags.csv
/content/drive/MyDrive/MoveLens/tag.csv


In [5]:
#data load
movie=pd.read_csv('/content/drive/MyDrive/MoveLens/movie.csv') 
rating=pd.read_csv('/content/drive/MyDrive/MoveLens/rating.csv')
link=pd.read_csv('/content/drive/MyDrive/MoveLens/link.csv')
tag=pd.read_csv('/content/drive/MyDrive/MoveLens/tag.csv') 
genome_tags=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_tags.csv') 
genome_scores=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_scores.csv')

In [6]:
#1. data 확인&활용

In [7]:
#유저 수
#rating.csv에서 겹치는 userId drop
userNum = rating["userId"].drop_duplicates().count()
print("유저 수:", userNum)

유저 수: 138493


In [8]:
#영화 수
#movie.csv에서 movieId count
movieNum = movie["movieId"].count()
print("영화 수:", movieNum)

영화 수: 27278


In [9]:
#실제 존재하는 rating의 수
#rating.csv에서 rating count
ratingNum = rating["rating"].count()
print("rating 수:", ratingNum)

rating 수: 20000263


In [10]:
#전체 rating 평균
#rating.csv에서 rating mean 계산
ratingMean = rating["rating"].mean()
print("rating 평균:", ratingMean)

rating 평균: 3.5255285642993797


In [11]:
#각 user rating 평균
#groupby(userId) 진행 후 mean 계산
userGroup = rating.groupby("userId")["rating"].agg(**{"mean_rating":"mean"})
userGroup.head()

#다른 방법
#userGroup = rating.groupby("userId")["rating"].mean()

,mean_rating
userId,
1,3.742857
2,4.000000
3,4.122995
4,3.571429
5,4.272727


In [12]:
#각 movie rating 평균
#group_by(movieId) 진행 후 mean 계산
# movieGroup = rating.groupby("movieId")["rating"].agg(**{"mean_rating":"mean"})
# movieGroup.count()

#movie 이름도 같이 출력할 경우
movieGroup = rating.groupby("movieId")["rating"].agg(**{"mean_rating":"mean"})
movieTitle = pd.merge(movieGroup, movie, on ="movieId", how="left").drop(columns="genres")
movieTitle = movieTitle.reindex(columns=['movieId', 'title', 'mean_rating'])
movieTitle.head()

,movieId,title,mean_rating
0,1,Toy Story (1995),3.921240
1,2,Jumanji (1995),3.211977
2,3,Grumpier Old Men (1995),3.151040
3,4,Waiting to Exhale (1995),2.861393
4,5,Father of the Bride Part II (1995),3.064592


In [13]:
#2. 각 csv 파일 설명

In [14]:
#movie.csv 확인
#데이터 : movieId(영화 ID), title(영화 제목), genres(영화 장르)
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
#rating.csv 확인
#데이터 : userId(유저 ID), movieId(영화 ID), rating(유저가 매긴 평점), timestamp(평가 시간)
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [16]:
#link.csv 확인
#데이터 : movieId(영화 ID), imdbID(IMDB에서 영화 ID), tmdbId(TMDB에서 영화 ID)
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [17]:
#tag.csv 확인
#데이터 : userId(유저 ID), movieId(영화 ID), tag(tag 명), timestamp(평가 시간)
tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [18]:
#genome_tags 확인
#데이터 : tagId(tag ID), tag(tag 명)
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [19]:
#genome_scores 확인
#데이터 : movieId(영화 ID), tagId(영화 tag), relevance(tag와의 유사도)
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [20]:
#3. RMSE 값

In [21]:
#필요한 모듈 import
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [22]:
#train/test data split 후 data 확인
train, test = train_test_split(rating, random_state=5)
 
print(train.shape)
print(test.shape)

(15000197, 4)
(5000066, 4)


In [26]:
#전체 rating 평균을 estimation할 때 RMSE
#train data의 mean 구하기
rating_mean = train['rating'].mean()
print("전체 rating 평균:", rating_mean)

#전체 rating의 mean: 평균 * 평가 데이터만큼의 리스트 생성
predictions = [rating_mean] * len(test)
 
#rmse 계산
mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
print("RMSE 값:", rmse)

전체 rating 평균: 3.525629030072072
RMSE 값: 1.0519912950635526


In [34]:
#user rating 평균을 estimation할 때 RMSE
#train data의 mean 구하기
rating_mean = train.groupby("userId")["rating"].mean().mean()
print("User rating 평균:", rating_mean)

#User rating의 mean: 평균 * 평가 데이터만큼의 리스트 생성
predictions = [rating_mean] * len(test)
 
#rmse 계산
mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
print("RMSE 값:", rmse)

User rating 평균: 3.627299879734278
RMSE 값: 1.0569315787127442


In [35]:
#movie rating 평균을 estimation할 때 RMSE
#train data의 mean 구하기
rating_mean = train.groupby("movieId")["rating"].mean().mean()
print("Movie rating 평균:", rating_mean)

#Movie rating의 mean: 평균 * 평가 데이터만큼의 리스트 생성
predictions = [rating_mean] * len(test)
 
#rmse 계산
mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
print("RMSE 값:", rmse)

Movie rating 평균: 3.142857790207713
RMSE 값: 1.1193265223808253
